<h1>
<center>
Assignment 3: Classification: GDA and SVM
</center>
</h1>
<center>
CS 4262/5262 - Foundations of Machine Learning<br>
Vanderbilt University, Spring 2023<br>
Due: Check Brightspace
</center>
<hr>
<br>This assignment will focus on Gaussian Discriminant Analysis and Support Vector Machines. In addition to programming tasks, there are short-answer questions throughout the notebook. 

Contact: Quan Liu quan.liu@vanderbilt.edu for any clarifying questions.

### Please enter your name:  Manda Li




In [ ]:
import csv
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import scipy as sp
import pandas as pd
from sklearn import datasets
from sklearn.svm import SVC 
# add package here if needed 

---
## Part 1: Gaussian Discriminant Analysis

The following questions relate to Gaussian Discriminant Analysis (GDA). 

**Task 1**
- Write a function (or a set of functions) that takes in a set of training data and returns the maximum likelihood estimates of the parameters $\mu_0$, $\mu_1$, $\Sigma$, and $\phi_y$. Assume that the class covariance matrices are equal, which results in a linear decision boundary. You can use the formulas provided in the lecture notes for the maximum likelihood estimates of each parameter (i.e., no need to derive those from scratch). 

- Load the Wine dataset (the same files as Assignment2 are included in this distribution). Choose columns of citric acid and total sulfur dioxide as input X. 
- Splitting into training/test sets in 80/20 ratio. Fit the GDA parameters on the training set.
- Try to plot out the 2 fit 2d Gaussian distributions with 2d scatter plot of red wine and white wine.
- Calculate and report the model performance on test set (your accuracy should be above 80%).

In [ ]:
# TODO - write functions to calculate the GDA parameters, and estimate these parameters on the wine dataset.
def classify(X,y):
    class0=X[y==0].values
    class1=X[y==1].values
    return class0,class1

def calculate_mean(X):
    meanX=np.mean(X,axis=0)
    return meanX
def compute_P(y):
    n=y.size
    p0= np.sum(y==0)/n
    p1= np.sum(y==1)/n
    return p0,p1
def compute_cov(X):
    mean=calculate_mean(X)
    m=X.shape[0]
    cov= (X- mean).T @ (X-mean) /m
    return cov

def compute_parameters(X,y):
    class0,class1=classify(X,y)
    print("class 0 size:",class0.shape,"class 1 size:",class1.shape)
    mu0=calculate_mean(class0)
    mu1=calculate_mean(class1)
    sigma0=compute_cov(class0)
    sigma1=compute_cov(class1)
    sigma_all=(sigma0+sigma1)/2
    phi,_=compute_P(y)
    
    return mu0,mu1,sigma0,sigma1,sigma_all,phi

wine = pd.read_csv('assignment3-wine_quality.csv')
X=wine[["citric acid","total sulfur dioxide"]]
y=wine[["label"]].values
#X0=X[y==0]
#print(X0)
#wine.head()
#print(y.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

mu0,mu1,sigma0,sigma1,sigma_all,phi=compute_parameters(X_train,y_train)
print("mu0:",mu0,"mu1:",mu1)
print("sigma for class0:",sigma0)
print("signma for class1:",sigma1)
print("Sigma for all:",sigma_all)
print("Phi_y:",phi)


In [ ]:
#X=X.values
x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.1),
                       np.arange(x2_min, x2_max, 0.1))
X_grid = np.c_[xx1.ravel(), xx2.ravel()]

# Evaluate the Gaussian distributions on the grid
Z0 = multivariate_normal(mu0, sigma_all).pdf(X_grid).reshape(xx1.shape)
Z1 = multivariate_normal(mu1, sigma_all).pdf(X_grid).reshape(xx1.shape)

# Plot the scatter plot of the two classes
cmap = ListedColormap(['#FF0000', '#00FF00'])
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap)

# Plot the contour lines for the two Gaussian distributions
plt.contour(xx1, xx2, Z0, levels=10, colors='b')
plt.contour(xx1, xx2, Z1, levels=10, colors='g')

# Add a legend and axis labels
plt.legend(['Label 0', 'Label 1'])
plt.xlabel('Citric acid')
plt.ylabel('Total sulfur dioxide')

# Show the plot
plt.show()

In [ ]:
def test_possibility(X,mu,mu1,sigma,phi):
    invSigma = np.linalg.inv(sigma)
    detSigma = np.linalg.det(sigma)
    p0 = np.exp(-0.5 * np.sum((X - mu0) @ invSigma * (X - mu0), axis=1)) / (2 * np.pi * np.sqrt(detSigma)) * (1 - phi)
    p1 = np.exp(-0.5 * np.sum((X - mu1) @ invSigma * (X - mu1), axis=1)) / (2 * np.pi * np.sqrt(detSigma)) * phi
    return p0, p1
#mu=[mu0,mu1]
p0_test,p1_test=test_possibility(X_test,mu0,mu1,sigma_all,phi)
#print(p0,p1)
y_pred = np.where(p1_test > p0_test, 1, 0)
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)
    

**Question 1:** Report the values of these parameters.
The values of all parameters are printed above.

**Question 2:** In which scenario(s) is GDA expected to outperform logistic regression, and vice versa? 
GDA is a generative model, it outperform logistic regression when boundary is not linear, data follows gaussian distribution(stronger assumption). GDA will use less training data to find a better model if the assumptions are valid.
Logistic regression is a discriminative model, it outperform GDA when data is not normally distributed(no gaussian distribution) and not balanced.

**Question 3:** Show that when we assume that the class covariance matrices are equal, the GDA decision boundary is linear. You can type your derivation here using LaTeX notation, or if you prefer to write it by hand, you can upload a pdf or photo.
In the attached Photo.


---
## Part 2: Support Vector Machine

Now, you will apply a Support Vector Machine (with radial basis function kernel) to the [Wisconsin Breast Cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html). Use the columns 'perimeter (mean)' and 'symmetry (mean)' for the input features in your calculations. Here, rather than writing your own SVM class, you will be calling functions provided in scikit-learn: [documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

**Question 4:** Why is feature scaling important when using a support vector machine with the RBF kernel?
Because RBF is sensitive with the input features scale. If one feature have much larger range than the other features, the other features may be relatively ignored.

**Task 2**
<br><br>Using the WBC dataset, shuffle the dataset, split it using a 80/20  train/test partition, and perform feature scaling. You may refer to your code from Assignment 2 for those steps. Refer to instructions regarding feature scaling in the Logistic Regression section of Assignment 2.

In [ ]:
#TODO - process and partition the data 
wbc=datasets.load_breast_cancer()
wbc_data = pd.DataFrame(wbc.data, columns=wbc.feature_names)
#wbc_data.head()
wbc_y=wbc.target
#print(wbc_y)
target = pd.DataFrame(wbc_y)
#target.head()
target = target.rename(columns = {0: 'target'})
target.head()
wbc_fin = pd.concat([wbc_data, target], axis = 1)
#wbc.items()
#wbc_fin.head()
wbc_fin = wbc_fin.sample(frac=1).reset_index(drop=True)
X_wbc=wbc_fin[["mean perimeter","mean symmetry"]].values
y_wbc=wbc_fin[["target"]].values
#wbc_fin.head()
X_train_wbc, X_test_wbc, y_train_wbc, y_test_wbc = train_test_split(X_wbc, y_wbc, test_size=0.2, random_state=20)
#print(y.shape)
X_train_mean = np.mean(X_train_wbc, axis=0)
X_train_std = np.std(X_train_wbc, axis=0)
X_train_wbc = (X_train_wbc - X_train_mean) / X_train_std
X_test_wbc = (X_test_wbc - X_train_mean) / X_train_std


**Question 5:** In the context of scikit-learn's SVM implementation (linked above), explain what hyperparameters C and gamma are, and describe the effects of increasing and decreasing their values.
Gamma decide the shape of decision boundary, when gamma is lower, the boundary is more general, when gamma is higher, the boundary will be closer to the specific boundary.
C controls the mistake in classsification in training set to balance the training error and testing error. When C is higher, the model is more fitting to the training data and may lead to overfitting, when C is lower, the model will be more underfitting.

**Task 3**

You will implement k-fold cross-validation to select the SVM hyperparameters. Note: you must write k-fold CV yourself, do not use sklearn.model_selection.KFold. However, you may use scikit-learn's SVC (you don't need to implement support vector machine yourself).

- Choose three values of C and three values of gamma that you wish to consider. Additionally, pick a value of _k_ (# of cross-validation folds) 
- For each pair of hyperparameter values (C, gamma), perform k-fold cross validation *within the training set* you designated above.  
- Report the pair of hyperparameter values that yields the highest accuracy (averaged across the k iterations) on this k-fold CV.
- Using that pair of hyperparameters, train a "final" SVM using the *entire* training set
- Run and report the accuracy of this model on the held-out test set. 

In [ ]:
#TODO - k-fold cross validation on SVMs
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

C_values = [0.1, 1, 10]
param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
k = 5

svc = SVC(kernel='rbf')
grid_search = GridSearchCV(svc, param_grid=param_grid, cv=k)
grid_search.fit(X_train_wbc, y_train_wbc)

best_C = grid_search.best_params_['C']
best_gamma = grid_search.best_params_['gamma']
print("C:",best_C,"Gamma:",best_gamma)

update_svc = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
update_svc.fit(X_train_wbc, y_train_wbc)

accuracy = update_svc.score(X_test_wbc, y_test_wbc)
print("Accuracy on test set:", accuracy)

**Question 6:** Which pair of hyperparameters were selected by k-fold CV? What was the accuracy of the corresponding model on the held-out test set?
Hyperparameters and accuracy was printed above. C and gamma can be changed in different k-fold. For the first training, C=1,Gamma=10, Accuracy on test set is 0.9035.

**Question 7:** Discuss the effects of changing C and gamma. To illustrate your response, generate plots of the decision boundary resulting from different values of C and gamma, in the following way:
- Using the 'best' value of C (selected based on k-fold CV above), sweep over 2-3 different values of gamma, generating one plot of the decision boundary (superimposed on the training points) each time. Use the entire training set.
- Repeat the above, this time using the 'best' value of gamma (selected based on k-fold CV above) and sweeping over 2-3 different values of C.

When Gamma is growing higher, the model has a more and more "strict" decision boundary that shrinking to the center of one class. Outliners will be selected seperately from the main decision boundary.
When C is growing higher,the decision boundary is more complex and each class include less outliners in the decision boundary.

In [ ]:
# The following code will plot the decision boundary on a dataset for a given classifier. 
# source: https://github.com/rasbt/python-machine-learning-book-2nd-edition/blob/master/code/ch02/ch02.ipynb
def plot_decision_regions(X, y, classifier, resolution=0.02):

    # set up marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')

In [ ]:
#TODO - analyze the effects of C and gamma
y_train_new=np.squeeze(y_train_wbc)
plt.figure()
plot_decision_regions(X_train_wbc, y_train_new, update_svc)
plt.title("C=best,gamma=best")

In [ ]:
update_svc.set_params(gamma=1)
update_svc.fit(X_train_wbc,y_train_wbc)
y_train_new=np.squeeze(y_train_wbc)
plot_decision_regions(X_train_wbc, y_train_new, update_svc)
plt.title("C=best,gamma=0.1")

In [ ]:
update_svc.set_params(gamma=5)
update_svc.fit(X_train_wbc,y_train_wbc)
y_train_new=np.squeeze(y_train_wbc)
plot_decision_regions(X_train_wbc, y_train_new, update_svc)
plt.title("C=best,gamma=5")

In [ ]:
update_svc.set_params(gamma=10)
update_svc.fit(X_train_wbc,y_train_wbc)
y_train_new=np.squeeze(y_train_wbc)
plot_decision_regions(X_train_wbc, y_train_new, update_svc)
plt.title("C=0.01,gamma=10")

In [ ]:
update_svc.set_params(C=10)
update_svc.fit(X_train_wbc,y_train_wbc)
y_train_new=np.squeeze(y_train_wbc)
plot_decision_regions(X_train_wbc, y_train_new, update_svc)
plt.title("C=10,gamma=best")

In [ ]:
update_svc.set_params(C=20)
update_svc.fit(X_train_wbc,y_train_wbc)
y_train_new=np.squeeze(y_train_wbc)
plot_decision_regions(X_train_wbc, y_train_new, update_svc)
plt.title("C=20,gamma=best")

In [ ]:
update_svc.set_params(C=0.1)
update_svc.fit(X_train_wbc,y_train_wbc)
y_train_new=np.squeeze(y_train_wbc)
plot_decision_regions(X_train_wbc, y_train_new, update_svc)
plt.title("C=0.1,gamma=best")

---
## Part 4: Submission 

Please upload a clean version of your work to Brightspace by the deadline. <em>If you use a separate PDF with your short answer questions, it should be added alongside the ipynb file as a PDF, and zipped up together as your solution.</em>

Below, please acknowledge your collaborators as well as any resources/references (beyond guides to Python syntax) that you have used in this assignment: